# Model Evaluation: LDA + BERT Hybrid Approach
This notebook evaluates a hybrid ML pipeline combining Latent Dirichlet Allocation (LDA) and BERT for information filtering and truth validation. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#df = pd.read_csv('../data/TFA_AlumniSurvey_2024.csv')
#df.head()

print("hello")

KeyboardInterrupt: 